# 3. Modell

In [10]:
!pip install pandas
!pip install openpyxl
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Datensatz laden (Sie müssen den Pfad anpassen damit es funktioniert) 
df = pd.read_excel("C:/Users/Cezar/Downloads/SpotifyTopSongs_1000.xlsx")

# Schritt 1:  Unwichtige nicht-numerische Spalten entfernen
columns_to_drop = ['spotify_id', 'name', 'artists', 'album_name']
df.drop(columns=columns_to_drop, inplace=True)

In [12]:

# Schritt 2: Kategorische Spalten codieren

categorical_columns = ['country', 'is_explicit']

# OneHotEncoder für nominale kategoriale Variablen verwenden
encoder = OneHotEncoder(drop='first', sparse_output=False)  
encoded_features = encoder.fit_transform(df[categorical_columns])

# Erstellen eines DataFrame für kodierte Merkmale
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_columns))

# Weglassen der ursprünglichen kategorialen Spalten und Verkettung der codierten Merkmale
df = df.drop(columns=categorical_columns)
df = pd.concat([df, encoded_df], axis=1)

In [13]:
# Schritt 3: Umgang mit date/time-Spalten
# Datumsspalten in numerische Merkmale umwandeln (year, month, day)
date_columns = ['snapshot_date', 'album_release_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])  
    df[f'{col}_year'] = df[col].dt.year
    df[f'{col}_month'] = df[col].dt.month
    df[f'{col}_day'] = df[col].dt.day
    df.drop(columns=[col], inplace=True)  # Originale Spalte löschen

In [14]:
# Schritt 4: Trennen Sie Merkmale (X) und Ziel (y)
# Angenommen, ‚daily_movement‘ ist die Zielvariable
X = df.drop(columns=['popularity'])
y = df['popularity']

In [15]:
# Schritt 5: Umgang mit fehlenden Werten (falls vorhanden)
X.fillna(X.mean(), inplace=True)  
y.fillna(y.mean(), inplace=True)  

In [16]:

# Schritt 6: Numerische Merkmale skalieren (optional für baumbasierte Modelle)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [17]:
# Schritt 7: Aufteilung der Daten in Trainings- und Testsätze
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Schritt 8: Das Modell RandomForestRegressor trainieren
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Vorhersage für Testmenge
y_pred = model.predict(X_test)

# Leistung des Modells bewerten
r2 = r2_score(y_test, y_pred)  # R² Leistung
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
rmse = mean_squared_error(y_test, y_pred)  # Root Mean Squared Error (RMSE)

# Bewertungsmetriken drucken
print("Model Performance:")
print(f"R^2 Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")


Model Performance:
R^2 Score: 0.6611
Mean Absolute Error (MAE): 6.7795
Mean Squared Error (MSE): 81.3018


### Dokumentation

#### 1. Datensatz aufteilen
Der Datensatz wurde in einen Trainingsdatensatz (80%) und einen Testdatensatz (20%) aufgeteilt.

#### 2. Algorithmus auswählen
Als Algorithmus wurde der **Random Forest Regressor** gewählt, da er robust ist und komplexe Beziehungen zwischen den Merkmalen und dem Ziel erfassen kann.

#### 3. Modell evaluieren
- **MAE:** {mae:.2f}
- **MSE:** {mse:.2f}
- **RMSE:** {rmse:.2f}
- **R² Score:** {r2:.2f}

#### 4. Fazit
Das Modell erreicht einen R²-Score von {r2:.2f}, was darauf hinweist, dass es {r2*100:.2f}% der Varianz im Zielattribut erklärt. Der MAE beträgt {mae:.2f}.

Nun exportieren wir noch das Model und unsere Daten

In [19]:
!pip install joblib

Defaulting to user installation because normal site-packages is not writeable


In [20]:
import joblib

joblib.dump(model, 'popularitaetsmodel.pkl')

['popularitaetsmodel.pkl']